<a href="https://colab.research.google.com/github/ShreyaTripathi1/IBM-GenAI-Project/blob/main/Code_Generation_from_Natural_Language_using_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving extended_programming_code_snippets.csv to extended_programming_code_snippets.csv


In [2]:
import pandas as pd

df = pd.read_csv("extended_programming_code_snippets.csv")
print(df.head())  # Show first few rows


                                            Query  \
0   How to create a list comprehension in Python?   
1           How to handle missing data in pandas?   
2         How to use a lambda function in Python?   
3              How to create a REST API in Flask?   
4  How to perform matrix multiplication in numpy?   

                                        Code_Snippet Language      Tags  
0                          [x**2 for x in range(10)]   Python  tutorial  
1                         df.fillna(0, inplace=True)   Python   example  
2                                    lambda x: x + 2   Python  advanced  
3  from flask import Flask\napp = Flask(__name__)...   Python  tutorial  
4                   import numpy as np\nnp.dot(A, B)   Python  advanced  


In [3]:
df['Query'] = df['Query'].str.lower()
df['Code_Snippet'] = df['Code_Snippet'].str.lower()
df['Tags'] = df['Tags'].str.lower()

In [4]:
df['Language'].value_counts()

,count
Language,
SQL,627
Java,618
Shell,601
JavaScript,595
Python,574
C++,5


In [5]:
from transformers import T5Tokenizer
# load t5 model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
# tokenize the dataset
def preprocess_data(examples):
  inputs = ["generate code: " + query for query in examples["Query"]]
  targets = examples["Code_Snippet"]
  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
  labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt").input_ids

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
from transformers import T5Tokenizer
# load t5 model
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Assuming 'df' is your DataFrame with 'Query' column
inputs = ["generate code: " + query for query in df['Query']]

# Now you can use the tokenizer
input_encodings = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")

In [7]:
from transformers import T5Tokenizer

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenize the dataset
def preprocess_data(data):
    inputs = ["generate code: " + query for query in data["Query"]]
    targets = data["Code_Snippet"].tolist()
    input_encodings = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    target_encodings = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    return input_encodings, target_encodings

input_encodings, target_encodings = preprocess_data(df)


In [8]:
import torch

class CodeSnippetDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

dataset = CodeSnippetDataset(input_encodings, target_encodings)

In [10]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Simplified TrainingArguments for older versions
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    do_train=True,
    do_eval=True,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreyatripathi1808 (shreyatripathi1808-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.058800
1000,0.212800
1500,0.084100
2000,0.048700
2500,0.030300
3000,0.027300
3500,0.024700


TrainOutput(global_step=3775, training_loss=0.1985125775368798, metrics={'train_runtime': 664.4637, 'train_samples_per_second': 22.725, 'train_steps_per_second': 5.681, 'total_flos': 510915300556800.0, 'train_loss': 0.1985125775368798, 'epoch': 5.0})

In [11]:
model.save_pretrained("/content/t5_finetuned")
tokenizer.save_pretrained("/content/t5_finetuned") # Fixed typo: tokenzier -> tokenizer

('/content/t5_finetuned/tokenizer_config.json',
 '/content/t5_finetuned/special_tokens_map.json',
 '/content/t5_finetuned/spiece.model',
 '/content/t5_finetuned/added_tokens.json')

In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your fine-tuned T5 model and tokenizer
model_path = "/content/t5_finetuned"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Set model to evaluation mode (important for inference)
model.eval()

def generate_code(prompt):
    """
    Generates code from a natural language prompt using the fine-tuned T5 model.

    Args:
        prompt (str): A natural language query describing the desired code.

    Returns:
        str: The generated Python code snippet.
    """
    prompt = prompt.strip().lower()
    input_text = f"generate code: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

    outputs = model.generate(
        input_ids,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Interactive loop for querying the model
print("🧠 Code Generator is ready! Type your request or 'exit' to quit.")

while True:
    user_input = input("\n🔎 Your Query:\n> ")

    if user_input.strip().lower() == "exit":
        print("\n👋 Goodbye!")
        break

    code = generate_code(user_input)

    print("\n💡 Generated Code:\n" + "-"*40)
    print(code)
    print("-"*40)


🧠 Code Generator is ready! Type your request or 'exit' to quit.

🔎 Your Query:
> create a function to check if a number is prime

💡 Generated Code:
----------------------------------------
check if a number is prime
----------------------------------------

🔎 Your Query:
> how to handle missing data in pandas?

💡 Generated Code:
----------------------------------------
df.fillna(0, inplace=true)
----------------------------------------

🔎 Your Query:
> exit

👋 Goodbye!
